## Simple Neural Network with NumPy

In [2]:
import numpy as np
import tensorflow as tf

# DataSet

I use function for genereating random data

In [3]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [4]:
X, y = load_coffee_data()
print(X.shape, y.shape)

(200, 2) (200, 1)


# Normalization of data
I normalaze data with tensorflow

In [5]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0])}, {np.min(X[:,0])}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1])}, {np.min(X[:,1])}")
norm_l = tf.keras.layers.Normalization(axis=1)
norm_l.adapt(X)
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0])}, {np.min(Xn[:,0])}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1])}, {np.min(Xn[:,1])}")

Temperature Max, Min pre normalization: 284.99434167453603, 151.32372212128612
Duration    Max, Min pre normalization: 15.454206928801511, 11.512747082472469
Temperature Max, Min post normalization: 1.659753680229187, -1.6852186918258667
Duration    Max, Min post normalization: 1.793238878250122, -1.6996712684631348


# Numpy Model

In [1]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Parameters
    ----------
    z : array_like
        A scalar or numpy array of any size.

    Returns
    -------
     g : array_like
         sigmoid(z)
    """
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

g = sigmoid # the activation function

In [21]:
def my_dense(a_in, W, b):
  units = W.shape[1]
  a_out = np.zeros(units)
  for j in range(units):
    w = W[:,j]
    z = np.dot(w, a_in) + b[j]
    a_out[j] = g(z)
  return a_out

def my_sequential(a, W1, b1, W2, b2):
  a1 = my_dense(a, W1, b1)
  a2 = my_dense(a1, W2, b2)
  return a2

In [22]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [23]:
def my_predict(X, W1, b1, W2, b2):
  m = X.shape[0]
  p = np.zeros((m, 1))
  for i in range(m):
    p[i, 0] = my_sequential(X[i], W1, b1, W2, b2)
  return p

We can try this routine on two examples:

In [24]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

<ipython-input-23-d4a19ace6461>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p[i, 0] = my_sequential(X[i], W1, b1, W2, b2)


In [26]:
yhat = np.zeros_like(predictions)

for i in range(len(predictions)):
  if predictions[i] >= 0.5:
    yhat[i] = 1
  else:
    yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]
